Celem ćwiczenia jest prezentacja możliwości wykorzystania metod uczenia maszynowego
w rozpoznawaniu komend głosowych.

Materiałem użytym w ćwiczeniu będą próbki nagrań komend z poprzednich ćwiczeń.
Przypomnijmy, w zbiorze jest:
- 30 (+szum) komend,
- nagranych ok. 1,7-2,4 tys. razy.

Treścią zadania jest klasyfikacja komend:
- klasa (class) = komenda,
- próbka (sample) = nagranie.

Ćwiczenie podzielone jest na cztery etapy:
1. Ekstrakcja cech z próbek
2. Podział próbek na treningowe i testowe
3. Uczenie klasyfikatora
4. Klasyfikacja nieznanej próbki
5. Ocena działania klasyfikatora


In [1]:
# import bibliotek
import os
import librosa
import pickle
import sklearn
import numpy

Optimal N_MFCC

In [2]:
# 1. Ekstrakcja cech z próbek
# Lista ekstraktorów cech: https://librosa.github.io/librosa/feature.html

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'C:/Users/Aayli/Documents/Delete'
commands = os.listdir(train_audio_path);  

# użyjemy MFCC, tj. Mel-frequency cepstral coefficients
# jednym z parametrów jest liczba wyjściowych współczynników:
n_mfcc = 40
qualities = []

# wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
for n_mfcc in range(1,101):
    classes = []
    class_size = 100 # użyjemy tylko po kilka danych
    samples = []
    labels = []
    for command in commands:
        if command == 'testing_list.txt' or command == 'validation_list.txt' or command == '_background_noise_' or command == 'tmp':
            continue
        classes.append(command)
        sample_num = 0
        for file in os.listdir(train_audio_path + '/' + command + '/'):
            sample_num += 1
            # przykład z MFCC:
            # y - waveform, sr - sampling rate, n_mfcc - number of MFCCs to return
            y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
            mfcc_seq = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = n_mfcc)
            mean_mfcc_seq = []
            for feature in mfcc_seq:
                mean_mfcc_seq.append(numpy.mean(feature))
            samples.append(mean_mfcc_seq)
            labels.append(command)
            if sample_num == class_size:
                break

    # warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq
    
    # 2. Wybór danych treningowych i testowych

    # Zautomatyzowany i losowy sposób to użycie:
    # "from sklearn.model_selection import train_test_split" oraz funkcji "train_test_split()"

    # wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
    # oraz kolejne 2 z każdej klasy jako testowe
    # uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
    train_size = 40
    test_size = 10
    train_samples = []
    train_labels = []
    test_samples = []
    test_labels = []
    for classname in classes:
        train_size_index = 0
        test_size_index = 0
        for i in range(len(labels)):
            if labels[i] == classname and train_size_index < train_size:
                train_samples.append(samples[i])
                train_labels.append(labels[i])
                train_size_index += 1
            elif labels[i] == classname and test_size_index < test_size:
                test_samples.append(samples[i])
                test_labels.append(labels[i])
                test_size_index += 1
            if train_size_index == train_size and test_size_index == test_size:
                break
            
    # proszę sprawdzić, czy dane zostały wybrane prawidłowo


    # 3. Uczenie klasyfikatora danymi treningowymi

    # Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
    # Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

    # skalowanie/normalizacja danych
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(train_samples)
    train_samples = scaler.transform(train_samples)
    test_samples = scaler.transform(test_samples)

    # uczenie
    from sklearn.neighbors import KNeighborsClassifier
    k = 5
    classifier = KNeighborsClassifier(n_neighbors = k)
    classifier.fit(train_samples, train_labels)


    # 4. Klasyfikacja nieznanych próbek
    predicted_labels = classifier.predict(test_samples)

    # zobaczmy wynik - zakomentować dla dużych zbiorów
    # print(test_labels)
    # print(predicted_labels)


    # 5. Ocena klasyfikatora
    # print(test_labels == predicted_labels)
    quality = numpy.sum(test_labels == predicted_labels) / len(test_labels) * 100;
    print(f'n_mfcc={n_mfcc} \t Ocena klasyfikatora: %.2f %%' % quality)
    qualities.append(quality)

n_mfcc_optimal = qualities.index(max(qualities))+1
print(f"best_mfcc = {n_mfcc_optimal} \t best_quality = {max(qualities)}")


n_mfcc=1 	 Ocena klasyfikatora: 2.67 %
n_mfcc=2 	 Ocena klasyfikatora: 5.00 %
n_mfcc=3 	 Ocena klasyfikatora: 4.00 %
n_mfcc=4 	 Ocena klasyfikatora: 6.00 %
n_mfcc=5 	 Ocena klasyfikatora: 6.67 %
n_mfcc=6 	 Ocena klasyfikatora: 6.00 %
n_mfcc=7 	 Ocena klasyfikatora: 9.33 %
n_mfcc=8 	 Ocena klasyfikatora: 8.67 %
n_mfcc=9 	 Ocena klasyfikatora: 11.33 %
n_mfcc=10 	 Ocena klasyfikatora: 8.33 %
n_mfcc=11 	 Ocena klasyfikatora: 8.00 %
n_mfcc=12 	 Ocena klasyfikatora: 8.67 %
n_mfcc=13 	 Ocena klasyfikatora: 7.00 %
n_mfcc=14 	 Ocena klasyfikatora: 6.67 %
n_mfcc=15 	 Ocena klasyfikatora: 6.67 %
n_mfcc=16 	 Ocena klasyfikatora: 8.67 %
n_mfcc=17 	 Ocena klasyfikatora: 8.67 %
n_mfcc=18 	 Ocena klasyfikatora: 9.67 %
n_mfcc=19 	 Ocena klasyfikatora: 9.33 %
n_mfcc=20 	 Ocena klasyfikatora: 10.00 %
n_mfcc=21 	 Ocena klasyfikatora: 8.67 %
n_mfcc=22 	 Ocena klasyfikatora: 8.00 %
n_mfcc=23 	 Ocena klasyfikatora: 8.00 %
n_mfcc=24 	 Ocena klasyfikatora: 7.33 %
n_mfcc=25 	 Ocena klasyfikatora: 6.33 %
n_mfcc=

Optimal train_size

In [7]:
# 1. Ekstrakcja cech z próbek
# Lista ekstraktorów cech: https://librosa.github.io/librosa/feature.html

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'C:/Users/Aayli/Documents/Delete'
commands = os.listdir(train_audio_path);  

# użyjemy MFCC, tj. Mel-frequency cepstral coefficients
# jednym z parametrów jest liczba wyjściowych współczynników:
n_mfcc = n_mfcc_optimal
qualities = []

# wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
for train_size in range(1,91):
    classes = []
    class_size = 100 # użyjemy tylko po kilka danych
    samples = []
    labels = []
    for command in commands:
        if command == 'testing_list.txt' or command == 'validation_list.txt' or command == '_background_noise_' or command == 'tmp':
            continue
        classes.append(command)
        sample_num = 0
        for file in os.listdir(train_audio_path + '/' + command + '/'):
            sample_num += 1
            # przykład z MFCC:
            # y - waveform, sr - sampling rate, n_mfcc - number of MFCCs to return
            y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
            mfcc_seq = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = n_mfcc)
            mean_mfcc_seq = []
            for feature in mfcc_seq:
                mean_mfcc_seq.append(numpy.mean(feature))
            samples.append(mean_mfcc_seq)
            labels.append(command)
            if sample_num == class_size:
                break

    # warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq

    # 2. Wybór danych treningowych i testowych

    # Zautomatyzowany i losowy sposób to użycie:
    # "from sklearn.model_selection import train_test_split" oraz funkcji "train_test_split()"

    # wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
    # oraz kolejne 2 z każdej klasy jako testowe
    # uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
#     train_size = 40
    test_size = 10
    train_samples = []
    train_labels = []
    test_samples = []
    test_labels = []
    for classname in classes:
        train_size_index = 0
        test_size_index = 0
        for i in range(len(labels)):
            if labels[i] == classname and train_size_index < train_size:
                train_samples.append(samples[i])
                train_labels.append(labels[i])
                train_size_index += 1
            elif labels[i] == classname and test_size_index < test_size:
                test_samples.append(samples[i])
                test_labels.append(labels[i])
                test_size_index += 1
            if train_size_index == train_size and test_size_index == test_size:
                break
            

    # 3. Uczenie klasyfikatora danymi treningowymi

    # Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
    # Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

    # skalowanie/normalizacja danych
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(train_samples)
    train_samples = scaler.transform(train_samples)
    test_samples = scaler.transform(test_samples)

    # uczenie
    from sklearn.neighbors import KNeighborsClassifier
    k = 5
    classifier = KNeighborsClassifier(n_neighbors = k)
    classifier.fit(train_samples, train_labels)


    # 4. Klasyfikacja nieznanych próbek
    predicted_labels = classifier.predict(test_samples)

    # zobaczmy wynik - zakomentować dla dużych zbiorów
    # print(test_labels)
    # print(predicted_labels)


    # 5. Ocena klasyfikatora
    # print(test_labels == predicted_labels)
    quality = numpy.sum(test_labels == predicted_labels) / len(test_labels) * 100;
    print(f'train_size={train_size} \t Ocena klasyfikatora: %.2f %%' % quality)
    qualities.append(quality)

train_size_optimal = qualities.index(max(qualities))+1
print(f"train_size = {train_size_optimal} \t best_quality = {max(qualities)}")


train_size=1 	 Ocena klasyfikatora: 2.33 %
train_size=2 	 Ocena klasyfikatora: 5.00 %
train_size=3 	 Ocena klasyfikatora: 4.67 %
train_size=4 	 Ocena klasyfikatora: 6.33 %
train_size=5 	 Ocena klasyfikatora: 4.67 %
train_size=6 	 Ocena klasyfikatora: 7.33 %
train_size=7 	 Ocena klasyfikatora: 8.33 %
train_size=8 	 Ocena klasyfikatora: 10.00 %
train_size=9 	 Ocena klasyfikatora: 8.67 %
train_size=10 	 Ocena klasyfikatora: 10.33 %
train_size=11 	 Ocena klasyfikatora: 14.33 %
train_size=12 	 Ocena klasyfikatora: 16.00 %
train_size=13 	 Ocena klasyfikatora: 13.33 %
train_size=14 	 Ocena klasyfikatora: 11.00 %
train_size=15 	 Ocena klasyfikatora: 10.67 %
train_size=16 	 Ocena klasyfikatora: 10.33 %
train_size=17 	 Ocena klasyfikatora: 8.67 %
train_size=18 	 Ocena klasyfikatora: 6.00 %
train_size=19 	 Ocena klasyfikatora: 6.00 %
train_size=20 	 Ocena klasyfikatora: 7.00 %
train_size=21 	 Ocena klasyfikatora: 7.67 %
train_size=22 	 Ocena klasyfikatora: 8.00 %
train_size=23 	 Ocena klasyfikato

Optiomal k

In [8]:
# 1. Ekstrakcja cech z próbek
# Lista ekstraktorów cech: https://librosa.github.io/librosa/feature.html

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'C:/Users/Aayli/Documents/Delete'
commands = os.listdir(train_audio_path);  

# użyjemy MFCC, tj. Mel-frequency cepstral coefficients
# jednym z parametrów jest liczba wyjściowych współczynników:
n_mfcc = n_mfcc_optimal
qualities = []

# wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
for k in range(1,11):
    classes = []
    class_size = 100 # użyjemy tylko po kilka danych
    samples = []
    labels = []
    for command in commands:
        if command == 'testing_list.txt' or command == 'validation_list.txt' or command == '_background_noise_' or command == 'tmp':
            continue
        classes.append(command)
        sample_num = 0
        for file in os.listdir(train_audio_path + '/' + command + '/'):
            sample_num += 1
            # przykład z MFCC:
            # y - waveform, sr - sampling rate, n_mfcc - number of MFCCs to return
            y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
            mfcc_seq = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = n_mfcc)
            mean_mfcc_seq = []
            for feature in mfcc_seq:
                mean_mfcc_seq.append(numpy.mean(feature))
            samples.append(mean_mfcc_seq)
            labels.append(command)
            if sample_num == class_size:
                break

    # warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq
    
    # 2. Wybór danych treningowych i testowych

    # Zautomatyzowany i losowy sposób to użycie:
    # "from sklearn.model_selection import train_test_split" oraz funkcji "train_test_split()"

    # wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
    # oraz kolejne 2 z każdej klasy jako testowe
    # uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
    train_size = train_size_optimal
    test_size = 10
    train_samples = []
    train_labels = []
    test_samples = []
    test_labels = []
    for classname in classes:
        train_size_index = 0
        test_size_index = 0
        for i in range(len(labels)):
            if labels[i] == classname and train_size_index < train_size:
                train_samples.append(samples[i])
                train_labels.append(labels[i])
                train_size_index += 1
            elif labels[i] == classname and test_size_index < test_size:
                test_samples.append(samples[i])
                test_labels.append(labels[i])
                test_size_index += 1
            if train_size_index == train_size and test_size_index == test_size:
                break
            

    # 3. Uczenie klasyfikatora danymi treningowymi

    # Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
    # Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

    # skalowanie/normalizacja danych
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(train_samples)
    train_samples = scaler.transform(train_samples)
    test_samples = scaler.transform(test_samples)

    # uczenie
    from sklearn.neighbors import KNeighborsClassifier
#     k = 5
    classifier = KNeighborsClassifier(n_neighbors = k)
    classifier.fit(train_samples, train_labels)


    # 4. Klasyfikacja nieznanych próbek
    predicted_labels = classifier.predict(test_samples)

    # zobaczmy wynik - zakomentować dla dużych zbiorów
    # print(test_labels)
    # print(predicted_labels)


    # 5. Ocena klasyfikatora
    # print(test_labels == predicted_labels)
    quality = numpy.sum(test_labels == predicted_labels) / len(test_labels) * 100;
    print(f'k={k} \t Ocena klasyfikatora: %.2f %%' % quality)
    qualities.append(quality)

k_optimal = qualities.index(max(qualities))+1
print(f"k = {k_optimal} \t best_quality = {max(qualities)}")


k=1 	 Ocena klasyfikatora: 17.00 %
k=2 	 Ocena klasyfikatora: 15.67 %
k=3 	 Ocena klasyfikatora: 17.00 %
k=4 	 Ocena klasyfikatora: 15.00 %
k=5 	 Ocena klasyfikatora: 16.00 %
k=6 	 Ocena klasyfikatora: 14.33 %
k=7 	 Ocena klasyfikatora: 13.67 %
k=8 	 Ocena klasyfikatora: 14.33 %
k=9 	 Ocena klasyfikatora: 13.33 %
k=10 	 Ocena klasyfikatora: 13.00 %
k = 1 	 best_quality = 17.0
